In [1]:
import os
import numpy as np
from tensorflow import keras
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [6]:
!pip install gdal

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\User\anaconda3\envs\tf\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-40rray2x\\gdal_0923c233f5d0438ca54ebbf786ad20e8\\setup.py'"'"'; __file__='"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-40rray2x\\gdal_0923c233f5d0438ca54ebbf786ad20e8\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\User\AppData\Local\Temp\pip-wheel-rw6q36ew'
       cwd: C:\Users\User\AppData\Local\Temp\pip-install-40rray2x\gdal_0923c233f5d0438ca54ebbf786ad20e8\
  Complete output (118 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\osgeo
  copying osgeo\gdal.py -> build\lib.win-amd64-3.8\osgeo
  copying osgeo\gd


  Running setup.py clean for gdal
Failed to build gdal
    Running setup.py install for gdal: started
    Running setup.py install for gdal: finished with status 'error'



  copying gdal-utils\osgeo_utils\samples\gdalident.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
  copying gdal-utils\osgeo_utils\samples\gdalimport.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
  copying gdal-utils\osgeo_utils\samples\gdalinfo.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
  copying gdal-utils\osgeo_utils\samples\gdallocationinfo.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
  copying gdal-utils\osgeo_utils\samples\gdal_auth.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
  copying gdal-utils\osgeo_utils\samples\gdal_cp.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
  copying gdal-utils\osgeo_utils\samples\gdal_create_pdf.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
  copying gdal-utils\osgeo_utils\samples\gdal_ls.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
  copying gdal-utils\osgeo_utils\samples\gdal_lut.py -> build\lib.win-amd64-3.8\osgeo_utils\samples
  copying gdal-utils\osgeo_utils\samples\gdal_mkdir.py -> build\lib.win-amd64-3.8\

In [ ]:
# Change the directory
os.chdir("E:\\BuiltUpPrediction")

In [2]:
# Assign file names
mxBangalore = 'l5_Bangalore2011_raw.tif'
builtupBangalore = 'l5_Bangalore2011_builtup.tif'
mxHyderabad = 'l5_Hyderabad2011_raw.tif'

In [3]:
# Read the rasters as array
ds1, featuresBangalore = raster.read(mxBangalore, bands='all')
ds2, labelBangalore = raster.read(builtupBangalore, bands=1)
ds3, featuresHyderabad = raster.read(mxHyderabad, bands='all')

In [4]:
# Print the size of the arrays
print("Bangalore Multispectral image shape: ", featuresBangalore.shape)
print("Bangalore Binary built-up image shape: ", labelBangalore.shape)
print("Hyderabad Multispectral image shape: ", featuresHyderabad.shape)

Bangalore Multispectral image shape:  (6, 2054, 2044)
Bangalore Binary built-up image shape:  (2054, 2044)
Hyderabad Multispectral image shape:  (6, 1318, 1056)


In [5]:
# Clean the labelled data to replace NoData values by zero
labelBangalore = (labelBangalore == 1).astype(int)

In [6]:
# Reshape the array to single dimensional array
featuresBangalore = changeDimension(featuresBangalore)
labelBangalore = changeDimension (labelBangalore)
featuresHyderabad = changeDimension(featuresHyderabad)
nBands = featuresBangalore.shape[1]

In [7]:
print("Bangalore Multispectral image shape: ", featuresBangalore.shape)
print("Bangalore Binary built-up image shape: ", labelBangalore.shape)
print("Hyderabad Multispectral image shape: ", featuresHyderabad.shape)

Bangalore Multispectral image shape:  (4198376, 6)
Bangalore Binary built-up image shape:  (4198376,)
Hyderabad Multispectral image shape:  (1391808, 6)


In [8]:
# Split testing and training datasets
xTrain, xTest, yTrain, yTest = train_test_split(featuresBangalore, labelBangalore, test_size=0.4, random_state=42)

In [9]:
print(xTrain.shape)
print(yTrain.shape)

(2519025, 6)
(2519025,)


In [10]:
print(xTest.shape)
print(yTest.shape)


(1679351, 6)
(1679351,)


In [11]:
# Normalise the data
xTrain = xTrain / 255.0
xTest = xTest / 255.0
featuresHyderabad = featuresHyderabad / 255.0

In [12]:
# Reshape the data
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
featuresHyderabad = featuresHyderabad.reshape((featuresHyderabad.shape[0], 1, featuresHyderabad.shape[1]))

In [14]:
# Print the shape of reshaped data
print(xTrain.shape, xTest.shape, featuresHyderabad.shape)

(2519025, 1, 6) (1679351, 1, 6) (1391808, 1, 6)


In [15]:
# Define the parameters of the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, nBands)),
    keras.layers.Dense(14, activation='relu'),
    keras.layers.Dense(2, activation='softmax')])

In [16]:
# Define the accuracy metrics and parameters
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=2)

# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]



Epoch 1/2
78720/78720 [==============================] - 313s 4ms/step - loss: 0.1337 - accuracy: 0.9512
Epoch 2/2
78720/78720 [==============================] - 277s 4ms/step - loss: 0.1142 - accuracy: 0.9579


In [17]:
# Calculate and display the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("Confusion matrix: for 14 nodes\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

predicted = model.predict(featuresHyderabad)
predicted = predicted[:,1]



Confusion matrix: for 14 nodes
 [[1458480   22396]
 [  47752  150723]]

P-Score: 0.871, R-Score: 0.759


In [18]:
# Predict new data and export the probability raster
prediction = np.reshape(predicted, (ds3.RasterYSize, ds3.RasterXSize))
outFile = 'Hyderabad_2011_BuiltupNN_predicted_1.tif'
raster.export(prediction, ds3, filename=outFile, dtype='float')